In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes # Разрешено для загрузки датасета
from sklearn.model_selection import train_test_split # Разрешено для разделения данных
from sklearn.metrics import mean_squared_error # Разрешено для оценки

# Импортируем нашу реализацию RBF Kernel Regression из предыдущего блока
# Предполагается, что класс SimpleRBFKernelRegression определен выше или в другом месте
# В рамках этого блока мы его повторно определим для самодостаточности.

class SimpleRBFKernelRegression:
    """
    Простая реализация Radial basis kernel regression
    с использованием Гауссовского (RBF) ядра.
    Использует только numpy и pandas.
    """
    def __init__(self, sigma=1.0, lambda_reg=0.0):
        """
        Инициализация Radial basis kernel regression.

        Args:
            sigma (float): Параметр ширины Гауссовского ядра. Контролирует радиус влияния
                           каждой обучающей точки. По умолчанию 1.0.
            lambda_reg (float): Параметр регуляризации (L2). Добавляет небольшую
                                регуляризацию для устойчивости. По умолчанию 0.0.
        """
        self.sigma = sigma
        self.lambda_reg = lambda_reg
        self.X_train = None
        self.y_train = None
        self._trained = False

    def _rbf_kernel(self, x1, x2):
        """
        Гауссовское (RBF) ядро.

        Args:
            x1 (np.ndarray): Первая точка(и).
            x2 (np.ndarray): Вторая точка(и).

        Returns:
            np.ndarray: Значение(я) ядра.
        """
        # Вычисляем squared Euclidean distance: ||x1 - x2||^2
        # Для двух векторов (1D): sum((x1 - x2)**2)
        # Для массива точек X1 и одной точки x2: sum((X1 - x2)**2, axis=1)
        # Для двух массивов точек X1 и X2 (матрица ядра):
        # K_ij = exp(-gamma * ||x_i - x_j||^2), gamma = 1 / (2 * sigma^2)
        # ||x_i - x_j||^2 = ||x_i||^2 + ||x_j||^2 - 2 * x_i.T * x_j

        # Убедимся, что x1 и x2 - numpy массивы
        x1 = np.asarray(x1)
        x2 = np.asarray(x2)

        # Вычисляем гамма
        gamma = 1.0 / (2 * self.sigma**2)

        # Если x1 и x2 - одномерные векторы (одна точка каждый)
        if x1.ndim == 1 and x2.ndim == 1:
            sq_dist = np.sum((x1 - x2)**2)
            return np.exp(-gamma * sq_dist)
        # Если x1 - массив точек, x2 - одна точка
        elif x1.ndim > 1 and x2.ndim == 1:
             # Используем broadcasting для вычисления разницы и квадрата
             sq_dist = np.sum((x1 - x2)**2, axis=1)
             return np.exp(-gamma * sq_dist)
        # Если x1 - одна точка, x2 - массив точек
        elif x1.ndim == 1 and x2.ndim > 1:
             sq_dist = np.sum((x1 - x2)**2, axis=1)
             return np.exp(-gamma * sq_dist)
        # Если x1 и x2 - массивы точек (вычисляем полную матрицу ядра)
        elif x1.ndim > 1 and x2.ndim > 1:
             # Более эффективный способ вычисления матрицы расстояний
             # ||x_i - x_j||^2 = ||x_i||^2 + ||x_j||^2 - 2 * x_i.T * x_j
             # sum(X1^2, axis=1)[:, None] - столбец сумм квадратов X1
             # sum(X2^2, axis=1)[None, :] - строка сумм квадратов X2
             # X1 @ X2.T - матрица скалярных произведений
             sq_dists = np.sum(x1**2, axis=1)[:, None] + np.sum(x2**2, axis=1)[None, :] - 2 * (x1 @ x2.T)
             # Избегаем отрицательных значений из-за ошибок округления
             sq_dists[sq_dists < 0] = 0
             return np.exp(-gamma * sq_dists)
        else:
             raise ValueError("Неподдержимые размерности для ядра")


    def fit(self, X, y):
        """
        Обучает модель Radial basis kernel regression.
        В этом методе мы просто сохраняем обучающие данные.

        Args:
            X (np.ndarray или pd.DataFrame): Входные признаки обучающих данных.
            y (np.ndarray или pd.Series): Целевая переменная обучающих данных.
        """
        self.X_train = np.asarray(X)
        self.y_train = np.asarray(y)
        self._trained = True

    def predict(self, X_test):
        """
        Прогнозирует значения для новых данных.

        Args:
            X_test (np.ndarray или pd.DataFrame): Входные признаки новых данных.

        Returns:
            np.ndarray: Спрогнозированные значения.
        """
        if not self._trained:
            raise RuntimeError("Модель не обучена. Вызовите fit сначала.")

        X_test = np.asarray(X_test)
        n_test_samples = X_test.shape[0]
        n_train_samples = self.X_train.shape[0]

        if n_train_samples == 0:
             return np.zeros(n_test_samples, dtype=float)

        # Вычисляем матрицу ядра между тестовыми и обучающими данными
        # K_test_train[i, j] = kernel(X_test[i], X_train[j])
        K_test_train = self._rbf_kernel(X_test, self.X_train)

        # Вычисляем матрицу ядра между обучающими данными (для регуляризации)
        K_train_train = self._rbf_kernel(self.X_train, self.X_train)

        # Добавляем регуляризацию к диагонали матрицы ядра обучающих данных
        # (K_train_train + lambda_reg * I)
        K_train_train_reg = K_train_train + self.lambda_reg * np.eye(n_train_samples)

        # Решаем систему линейных уравнений для нахождения весов (альфа)
        # (K_train_train + lambda_reg * I) * alpha = y_train
        # alpha = inv(K_train_train + lambda_reg * I) * y_train
        try:
            alpha = np.linalg.solve(K_train_train_reg, self.y_train)
        except np.linalg.LinAlgError:
            # Если матрица вырожденная, используем псевдоинверсию (менее устойчиво)
            print("Предупреждение: Матрица ядра плохо обусловлена. Использование псевдоинверсии.")
            alpha = np.linalg.pinv(K_train_train_reg) @ self.y_train


        # Вычисляем предсказания
        # y_pred = K_test_train @ alpha
        predictions = K_test_train @ alpha

        return predictions


# 1. Загрузка реального датасета (Diabetes)
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# 2. Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape[0]} примеров")
print(f"Размер тестовой выборки: {X_test.shape[0]} примеров")
print(f"Количество признаков: {X_train.shape[1]}")


# 3. Применение нашей реализации RBF Kernel Regression
# Подбираем гиперпараметры (sigma и lambda_reg) - это важный шаг в реальных задачах.
# Здесь используем фиксированные значения для примера.
# В реальной задаче нужно использовать кросс-валидацию для подбора.
my_rbfkr = SimpleRBFKernelRegression(sigma=0.5, lambda_reg=0.1)

# Обучение модели
my_rbfkr.fit(X_train, y_train)

# Предсказание на тестовой выборке
my_predictions = my_rbfkr.predict(X_test)

# 4. Оценка производительности на тестовой выборке
my_mse_test = mean_squared_error(y_test, my_predictions)
print(f"\nMSE нашей реализации RBF Kernel Regression на тестовой выборке: {my_mse_test:.4f}")

# Опционально: Сравнение с sklearn SVR с RBF ядром
# try:
#     from sklearn.svm import SVR
#
#     # Подбираем гиперпараметры для SVR (C, gamma)
#     # gamma для SVR аналогичен 1 / (2 * sigma^2)
#     sklearn_svr = SVR(kernel='rbf', C=100.0, gamma=1.0/(2*my_rbfkr.sigma**2))
#     sklearn_svr.fit(X_train, y_train)
#     sklearn_predictions = sklearn_svr.predict(X_test)
#     sklearn_mse_test = mean_squared_error(y_test, sklearn_predictions)
#     print(f"MSE sklearn SVR (RBF) на тестовой выборке: {sklearn_mse_test:.4f}")
#
# except ImportError:
#     print("\nБиблиотека sklearn не найдена для сравнения SVR.")
# except Exception as e:
#     print(f"\nОшибка при сравнении со sklearn SVR: {e}")

